In [1]:
import torchvision.transforms as T
import torch
import os
from PIL import Image

def to_latent_and_back(filename, device="mps"):
    transform = T.Compose([
        T.Resize(512, antialias=True),
        # This will center crop to 512x512 from the resized image
        T.CenterCrop(512),
        T.ToTensor(),
        T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        lambda x: x.to(dtype=torch.float16)
    ])
    image = Image.open(filename)
    image = transform(image)[None].to(device)
    
    # Encode and decode
    latent = dc_ae.encode(image)
    image_out = dc_ae.decode(latent.latent)

    return [image * 0.5 + 0.5, image_out.sample * 0.5 + 0.5]

def concatenate_images(image_paths, output_path):
    images = [Image.open(image_path) for image_path in image_paths]
    widths, heights = zip(*(i.size for i in images))

    total_width = sum(widths)
    max_height = max(heights)

    new_image = Image.new('RGB', (total_width, max_height))

    x_offset = 0
    for image in images:
        new_image.paste(image, (x_offset, 0))
        x_offset += image.width

    new_image.save(output_path)

In [2]:
from diffusers import AutoencoderDC

device = "mps"
dc_ae = AutoencoderDC.from_pretrained("mit-han-lab/dc-ae-f64c128-in-1.0-diffusers", torch_dtype=torch.float16).to(device).eval()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [4]:
import os
from torchvision.utils import save_image

inputs_path, output_path, outputgall_path = "./inputs", "./output", "./output_galleries"

for directory in [output_path, outputgall_path]:
    if not os.path.exists(directory): os.makedirs(directory)

for image_filename in os.listdir(inputs_path):
    print(image_filename)
    input_transformed, latent_decoded = to_latent_and_back(inputs_path + "/" + image_filename, device)
    base_path, ext = os.path.splitext(image_filename)
    save_image(input_transformed, f"{output_path}/{base_path}_input{ext}")
    save_image(latent_decoded, f"{output_path}/{base_path}_decoded{ext}")
    concatenate_images([f"{output_path}/{base_path}_input{ext}", f"{output_path}/{base_path}_decoded{ext}"],f"{outputgall_path}/{base_path}_gallery{ext}")


bafkreicrlzkhyfldaa57g5cutelqakie2yot2mykr7wrabowmam3bznnoq.png
bafkreieczeyp3ccd3rnt4jsahwlqxvxf4eicsbvzfhg2iwwufmxwl7f7b4.png
bafkreied4rpb2ef2r6btzqqrjksfgwaylcpliivpoxy6sfg6xu767qwouq.png
bafkreidciyfpf43nnh5qqskflxtpnvcqjqs4myjuvgybohxmtdujrlr36m.jpg
bafkreiakrdiro2y2q4adb6yxks777fbbpv2dppfzi57l5sbizz24qetsna.jpg
bafkreicwgvjwdxgxt3awgnf7psmavhfpy2hebeaqzsuc7efdn4fdh2jto4.jpg
bafkreihwgl3mrmcvpz2xa6njkv4qjr2yhy3wyniy4r3i3pmvmwnwffbdkq.png
bafkreiez2thdzw5eti7l7guqdw7mjn63jgebqlxtzsiy2ldqgg3whlbyjm.png
bafkreid2ljsslwvqaa64nva4q4sowdp72mtdvjdj63amiifcauf7rmcu6q.png
